## 1. 環境セットアップ

In [ ]:
# Google Colab環境かどうかを確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✅ Running in local environment")

In [ ]:
# Google Colab環境の場合、必要なパッケージをインストール
if IN_COLAB:
    print("📦 Installing required packages...")
    !pip install -q transformer-lens sae-lens einops jaxtyping datasets huggingface_hub
    print("✅ Installation completed")
else:
    print("ℹ️  Skipping package installation (local environment)")

## 2. Google Driveのマウント（Colab環境の場合）

In [ ]:
import os

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

    # プロジェクトディレクトリに移動
    project_path = '/content/drive/MyDrive/sae_pj2'
    os.chdir(project_path)
    print(f"✅ Changed directory to: {project_path}")
else:
    print("ℹ️  Skipping Google Drive mount (local environment)")

print(f"📂 Current directory: {os.getcwd()}")

## 3. モジュールのインポートと設定

In [ ]:
import sys
import torch
from pathlib import Path

# プロジェクトのルートディレクトリをパスに追加
if IN_COLAB:
    sys.path.append('/content/drive/MyDrive/sae_pj2')

# 自作モジュールをインポート
from intervention_runner import InterventionRunner
from config import INTERVENTION_GEMMA2_9B_IT_CONFIG

print("✅ Modules imported successfully")

# GPUの確認
if torch.cuda.is_available():
    print(f"🎮 GPU available: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ No GPU available. Using CPU (this will be slow)")

## 4. 介入対象の特徴量IDを指定

Step 4 のフィルタリングで特定された迎合性特徴量のIDリストを指定します。

**例**: Attribution Patching の結果から、上位20個の特徴量を選択した場合

In [ ]:
# ============================================================
# 【重要】ここに介入対象の特徴量IDリストを指定してください
# ============================================================

# 例: Step 4 で特定された上位20個の迎合性特徴量
# intervention_feature_ids = [
#     123, 456, 789, 1024, 2048,  # 実際のIDに置き換えてください
#     3072, 4096, 5120, 6144, 7168,
#     8192, 9216, 10240, 11264, 12288,
#     13312, 14336, 15360, 16000, 16383
# ]

# または、csvファイルから読み込む場合:
# with open('results/selected_features.json', 'r') as f:
#     data = json.load(f)
#     intervention_feature_ids = data['top_k_features']
import pandas as pd

df = pd.read_csv("results/intervention_candidates_20251211_193847.csv")
intervention_feature_ids = list(df["feature_index"])

print(f"🎯 Intervention target: {len(intervention_feature_ids)} features")
print(f"   Feature IDs: {intervention_feature_ids[:5]}... (showing first 5)")

## 5. 実験設定の確認と調整

必要に応じて、実験パラメータを調整できます。

In [ ]:
# 設定をコピーして調整（オプション）
from copy import deepcopy
from config import DataConfig, GenerationConfig

# デフォルトの設定を使用
config = INTERVENTION_GEMMA2_9B_IT_CONFIG

# サンプル数を調整したい場合（テスト実行用）
# config.data.sample_size = 5  # 例: 最初の5問のみで実験

# 生成パラメータを調整したい場合
# config.generation.temperature = 0.5
# config.generation.max_new_tokens = 100

print("⚙️  Experiment Configuration:")
print(f"   Model: {config.model.name}")
print(f"   SAE: {config.model.sae_release}/{config.model.sae_id}")
print(f"   Hook: {config.model.hook_name}")
print(f"   Sample size: {config.data.sample_size}")
print(f"   Max new tokens: {config.generation.max_new_tokens}")
print(f"   Temperature: {config.generation.temperature}")
print(f"   Dataset: {config.data.dataset_path}")

In [ ]:
from huggingface_hub import login
login()  # 画面の指示に従いトークン入力

## 6. 介入実験の実行

InterventionRunnerを初期化して実験を実行します。

In [ ]:
# InterventionRunnerの初期化
runner = InterventionRunner(
    config=config,
    intervention_feature_ids=intervention_feature_ids
)

print("✅ InterventionRunner initialized")

### オプション: データ範囲を指定して実験を実行

大量のデータを処理する場合、範囲を指定して段階的に実行できます。

In [ ]:
# 実験実行のパターン

# パターン1: 設定で指定したsample_size分だけ実行
output_path = runner.run_complete_experiment(start_index=1200, end_index=1300)

# パターン2: sample_sizeを直接指定
# output_path = runner.run_complete_experiment(sample_size=10)

# パターン3: インデックス範囲を指定（0-based）
# 例: 101問目から200問目まで (index 100-199)
# output_path = runner.run_complete_experiment(start_index=100, end_index=200)

print(f"\n✅ Experiment completed!")
print(f"📁 Results saved to: {output_path}")

## 7. 結果の確認

保存された結果ファイルを読み込んで、簡単な確認を行います。

In [ ]:
import json

# 結果ファイルを読み込み
with open(output_path, 'r', encoding='utf-8') as f:
    results = json.load(f)

print("📊 Experiment Results Summary:")
print("="*60)
print(f"Model: {results['metadata']['model_name']}")
print(f"Intervention method: {results['metadata']['intervention_method']}")
print(f"Number of intervention features: {results['metadata']['num_intervention_features']}")
print(f"Total questions processed: {results['metadata']['num_questions']}")
print(f"Question ID range: {results['metadata']['question_id_range']['start']} - {results['metadata']['question_id_range']['end']}")
print(f"Timestamp: {results['metadata']['timestamp']}")
print("="*60)

### サンプル結果の表示

In [ ]:
# 最初の質問の結果を詳しく表示
if results['results']:
    first_question = results['results'][0]

    print("\n📝 Sample Result (Question 1):")
    print("="*60)
    print(f"Dataset: {first_question['dataset']}")
    print(f"Base text: {first_question['base_text'][:100]}...")
    print(f"\nNumber of variations: {len(first_question['variations'])}")

    # 最初のバリエーションの詳細
    first_variation = first_question['variations'][0]
    print(f"\n--- Variation 1: {first_variation['template']} ---")
    print(f"Prompt: {first_variation['prompt'][:150]}...")
    print(f"\nBaseline Response:")
    print(f"  {first_variation['baseline_response']}")
    print(f"\nIntervention Response:")
    print(f"  {first_variation['intervention_response']}")
    print("="*60)
else:
    print("⚠️ No results found")


## 8. 応答の比較分析（簡易版）

BaselineとInterventionの応答が異なるケースをカウントします。

In [ ]:
# 応答が変化したケースをカウント
total_variations = 0
changed_variations = 0

for question in results['results']:
    for variation in question['variations']:
        total_variations += 1
        baseline = variation['baseline_response'].strip()
        intervention = variation['intervention_response'].strip()

        if baseline != intervention:
            changed_variations += 1

change_rate = (changed_variations / total_variations * 100) if total_variations > 0 else 0

print("\n📈 Response Change Analysis:")
print("="*60)
print(f"Total variations processed: {total_variations}")
print(f"Variations with changed response: {changed_variations}")
print(f"Change rate: {change_rate:.2f}%")
print("="*60)
print("\nℹ️  Next steps:")
print("  1. Use GPT-4o to evaluate sycophancy flags for baseline vs intervention")
print("  2. Use GPT-4o to rate naturalness scores (1-5 scale)")
print("  3. Perform McNemar's test for statistical significance")
print("  4. Analyze qualitative changes in response content")

## 次のステップ

実験結果を用いて、以下の評価を行います：

1. **Sycophancy Rate (迎合率) の評価**
   - GPT-4o を用いて Baseline と Intervention の応答に対して迎合フラグを判定
   - 迎合率の変化を計算
   - McNemar検定で統計的有意性を検証

2. **Naturalness Score (自然さスコア) の評価**
   - GPT-4o を用いて1-5のリッカート尺度で評価
   - 介入による自然さの低下がないか確認

3. **定性分析**
   - 応答がどのように変化したかを詳しく分析
   - 特に効果的だったケースと効果がなかったケースを比較

これらの評価は別のノートブックまたはスクリプトで実施することを推奨します。